#### Auto labelization with yolo

In [2]:
# Load the images from the folder "../images"
import os

# Load the images from the folder "../images"
images = []
for filename in os.listdir("../output/images"):
    images.append(filename)


# Download and place it into the folder "../config"
# https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
# https://pjreddie.com/media/files/yolov3.weights
# https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names

def download(url, filename):
    import functools
    import pathlib
    import shutil
    import requests
    from tqdm.auto import tqdm

    r = requests.get(url, stream=True, allow_redirects=True)
    if r.status_code != 200:
        r.raise_for_status()  # Will only raise for 4xx codes, so...
        raise RuntimeError(f"Request to {url} returned status code {r.status_code}")
    file_size = int(r.headers.get('Content-Length', 0))

    path = pathlib.Path(filename).expanduser().resolve()
    path.parent.mkdir(parents=True, exist_ok=True)

    desc = "(Unknown total file size)" if file_size == 0 else ""
    r.raw.read = functools.partial(r.raw.read, decode_content=True)  # Decompress if needed
    with tqdm.wrapattr(r.raw, "read", total=file_size, desc=desc) as r_raw:
        with path.open("wb") as f:
            shutil.copyfileobj(r_raw, f)

    return path


# Check if exist the folder "../images_labelized"
import os

if not os.path.exists("../output/images_labelized"):
    os.makedirs("../output/images_labelized")

# Check if the folder "../labels" exist
if not os.path.exists("../output/labels"):
    os.makedirs("../output/labels")

# check if exist the folder "../config"
if not os.path.exists("../output/config"):
    os.makedirs("../output/config")

print("All folders are created")

# Check if the file yolov3.cfg exist
if not os.path.isfile("../output/config/yolov3.cfg"):
    download("https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg", "../output/config/yolov3.cfg")

# Check if the file yolov3.weights exist
if not os.path.isfile("../output/config/yolov3.weights"):
    download("https://pjreddie.com/media/files/yolov3.weights", "../output/config/yolov3.weights")

# Check if the file coco.names exist
if not os.path.isfile("../output/config/coco.names"):
    download("https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names", "../output/config/coco.names")

print("All files are downloaded")

All folders are created
All files are downloaded


In [3]:
import threading
from multiprocessing import Pool
import concurrent
from tqdm import tqdm
import pickle
import numpy as np
import cv2
import os


def detect(
        path_name,
        output_folder = "../output/images_labelized/",
        label_path = "../output/config/coco.names",
        weights_path = "../output/config/yolov3.weights",
        config_path = "../output/config/yolov3.cfg",
        CONFIDENCE = 0.5,
        SCORE_THRESHOLD = 0.5,
        IOU_THRESHOLD = 0.5
):
    labels = open(label_path).read().strip().split("\n")
    colors = np.random.randint(0, 255, size=(len(labels), 3), dtype="uint8")

    net = cv2.dnn.readNetFromDarknet(config_path, weights_path)

    image = cv2.imread(path_name)
    file_name = os.path.basename(path_name)
    filename, ext = file_name.split(".")

    h, w = image.shape[:2]
    # create 4D blob
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)

    # sets the blob as the input of the network
    net.setInput(blob)
    # get all the layer names
    ln = net.getLayerNames()
    try:
        ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    except IndexError:
        # in case getUnconnectedOutLayers() returns 1D array when CUDA isn't available
        ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]
    # feed forward (inference) and get the network output
    # measure how much it took in seconds
    layer_outputs = net.forward(ln)

    font_scale = 1
    thickness = 1
    boxes, confidences, class_ids = [], [], []
    text_labels = []
    # loop over each of the layer outputs
    for output in layer_outputs:
        # loop over each of the object detections
        for detection in output:
            # extract the class id (label) and confidence (as a probability) of
            # the current object detection
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            # discard out weak predictions by ensuring the detected
            # probability is greater than the minimum probability
            if confidence > CONFIDENCE:
                # scale the bounding box coordinates back relative to the
                # size of the image, keeping in mind that YOLO actually
                # returns the center (x, y)-coordinates of the bounding
                # box followed by the boxes' width and height
                box = detection[:4] * np.array([w, h, w, h])
                (centerX, centerY, width, height) = box.astype("int")
                # use the center (x, y)-coordinates to derive the top and
                # and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                # update our list of bounding box coordinates, confidences,
                # and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                class_ids.append(class_id)

        # loop over the indexes we are keeping
        for i in range(len(boxes)):
            text_labels.append(labels[class_ids[i]])
            # extract the bounding box coordinates
            x, y = boxes[i][0], boxes[i][1]
            w, h = boxes[i][2], boxes[i][3]
            # draw a bounding box rectangle and label on the image
            color = [int(c) for c in colors[class_ids[i]]]
            cv2.rectangle(image, (x, y), (x + w, y + h), color=color, thickness=thickness)
            text = f"{labels[class_ids[i]]}: {confidences[i]:.2f}"
            # calculate text width & height to draw the transparent boxes as background of the text
            (text_width, text_height) = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, fontScale=font_scale, thickness=thickness)[0]
            text_offset_x = x
            text_offset_y = y - 5
            box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 2, text_offset_y - text_height))
            overlay = image.copy()
            cv2.rectangle(overlay, box_coords[0], box_coords[1], color=color, thickness=cv2.FILLED)
            # add opacity (transparency to the box)
            image = cv2.addWeighted(overlay, 0.6, image, 0.4, 0)
            # now put the text (label: confidence %)
            cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale=font_scale, color=(0, 0, 0), thickness=thickness)
    return text_labels

    #cv2.imwrite(os.path.join(output_folder, filename + "." + ext), image)



def update_tags(images):
    # Run the YOLOv3 algorithm on each image
    # display progress bar in the first thread only
    for image in tqdm(images, desc="Updating tags"):
        # read pickle file from ../output/metadata/file_name.pkl
        file_name = os.path.basename(image)
        file_name, ext = file_name.split(".")
        with open(os.path.join("../output/metadata", file_name + ".pickle"), "rb") as f:
            metadata = pickle.load(f)
        
        if "tags" in metadata:
            continue

        labels = detect(image)

        # Remove duplicates from labels
        labels = list(set(labels))
        # add labels to metadata
        metadata["tags"] = labels
        # save metadata to pickle file
        with open(os.path.join("../output/metadata", file_name + ".pickle"), "wb") as f:
            pickle.dump(metadata, f)
        

# Get the list of images
images = os.listdir("../output/images")
images = [os.path.join("../output/images", image) for image in images]

update_tags(images)





### Now, find dominant colors in the images

In [ ]:
import cv2


# Convert RGB to HEX (RGB is an array of 3 values)
def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % (int(rgb[0]), int(rgb[1]), int(rgb[2]))


# Find 4 dominant colors in the image
def find_dominant_colors(image, k=4, image_processing_size=None):
    # Need to return a list of 4 colors
    assert k <= 4, "k needs to be less than or equal to 4"
    # Resize image if new dims provided, just to speed up processing
    if image_processing_size is not None:
        image = cv2.resize(image, image_processing_size, interpolation=cv2.INTER_AREA)
    # Convert to RGB from BGR
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Reshape the image to be a list of pixels
    image = image.reshape((image.shape[0] * image.shape[1], 3))
    # Cluster and assign labels to the pixels
    clt = KMeans(n_clusters=k, n_init=10)
    labels = clt.fit_predict(image)
    # Count labels to find most popular
    label_counts = Counter(labels)
    # Subset out most popular centroid
    dominant_colors = [clt.cluster_centers_[i] for i in label_counts.keys()]
    # For each color, convert to hex and append to list
    dominant_colors = [rgb_to_hex(color) for color in dominant_colors]

    # find percent of each color in the image
    percent = [int((label_counts[i] / len(labels)) * 100) for i in label_counts.keys()]

    # return the 4 dominant colors and their percent
    dominant_colors = list(zip(dominant_colors, percent))
    return dominant_colors


# For each image in the folder "../images_labelized"
for image in tqdm(images, desc="Finding dominant colors"):
    img = cv2.imread("../output/images/" + image)
    dominant_color = find_dominant_colors(img)
    # save the image in the folder "../images_labelized"
    with open("../output/labels/" + image.split(".")[0] + ".txt", "a") as f:
        f.write(f"dominant_color:{dominant_color}")


In [ ]:
# Create histogram of the dominant colors
import matplotlib.pyplot as plt
import numpy as np

# For each image in the folder "../images_labelized" get the line that contains the dominant color
for image in tqdm(images, desc="Creating color pie charts"):
    with open("../output/labels/" + image.split(".")[0] + ".txt", "r") as f:
        for line in f:
            if "dominant_color" in line:
                # Get the dominant color
                dominant_color = line.split(":")[1]
                dominant_color = eval(dominant_color)
                # Create the pie chart
                labels = [i[0] for i in dominant_color]
                sizes = [i[1] for i in dominant_color]
                fig1, ax1 = plt.subplots()
                # change the colors of the pie chart
                colors = [i[0] for i in dominant_color]
                ax1.pie(sizes, colors=colors, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)
                ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
                # Save the pie chart in the folder "../histogram"
                plt.savefig("../output/histogram/" + image.split(".")[0] + ".png")
                plt.close()


# Group the images by multiple data
# there is multiple ways to group the images (in the folder labels, we can find the label, the dominant color), in metadata folder, we have the metadata of each image

In [ ]:
import pandas as pd
import pickle

# Create a dataframe with the metadata of each image, one pikle file per image
df = pd.DataFrame(columns=["image", "label", "dominant_color", "metadata"])

# For each image in the folder "../images_labelized" get the line that contains the dominant color
for image in tqdm(images, desc="Creating dataframe"):
    # Example of lines :
    # label:person,posX:113,posY:120,width:134,height:360,confidence:0.9987273216247559%
    # label:horse,posX:325,posY:187,width:282,height:270,confidence:0.9985314607620239%
    # label:handbag,posX:143,posY:385,width:82,height:91,confidence:0.5130287408828735%
    # dominant_color:[('#3e4542', 37), ('#6d6f6d', 22), ('#a6a4a7', 9), ('#191b1d', 30)]

    # Get the label
    with open("../output/labels/" + image.split(".")[0] + ".txt", "r") as f:
        # it's possible to have more than one label per image
        label = ""
        dominant_color = ""

        for line in f:
            if "label" in line:
                raw_label = line.split(":")[1]
                # cut as the first , is the end of the label
                label += raw_label.split(",")[0]
                label += ","
            if "dominant_color" in line:
                dominant_color = line.split(":")[1]
                dominant_color = eval(dominant_color)
                dominant_color = [i[0] for i in dominant_color]
                dominant_color = ",".join(dominant_color)
        # Get the metadata
        with open("../output/metadata/" + image.split(".")[0] + ".pickle", "rb") as f:
            metadata = pickle.load(f)
            # Convert metadata to string : key:value,key:value
            metadata = ",".join([f"{key}:{value}" for key, value in metadata.items()])

        # Add the image to the dataframe by using panda.concat
        df = pd.concat([df, pd.DataFrame([[image, label, dominant_color, metadata]],
                                         columns=["image", "label", "dominant_color", "metadata"])])

# Save the dataframe
df.to_csv("../output/dataframe.csv", index=False)


In [ ]:
# Group the images by label
import os

# Create a folder for each label
for label in df["label"].unique():
    if not os.path.exists("../output/grouped_by_label/" + label):
        os.makedirs("../output/grouped_by_label/" + label)

# For each image in the folder "../images_labelized" get the line that contains the dominant color
for image in tqdm(images, desc="Grouping images by label"):
    # Get the label
    with open("../output/labels/" + image.split(".")[0] + ".txt", "r") as f:
        for line in f:
            if "label" in line:
                label = line.split(":")[1]
                label = label.split(",")[0]
    # Copy the image in the folder "../grouped_by_label"
    os.system(f"cp ../output/images/{image} ../output/grouped_by_label/{label}/{image}")

In [ ]:
# Group image by dominant color, take the color one by one and create a folder for each color
for colors in df["dominant_color"].unique():
    # split colors by , to get individual colors
    colors = colors.split(",")
    for color in colors:
        if not os.path.exists("../output/grouped_by_dominant_color/" + color):
            os.makedirs("../output/grouped_by_dominant_color/" + color)

# For each image in the folder "../images_labelized" get the line that contains the dominant color
for image in tqdm(images, desc="Grouping images by dominant color"):
    # Get the dominant color
    with open("../output/labels/" + image.split(".")[0] + ".txt", "r") as f:
        for line in f:
            if "dominant_color" in line:
                dominant_color = line.split(":")[1]
                dominant_color = eval(dominant_color)
                dominant_color = [i[0] for i in dominant_color]
                dominant_color = ",".join(dominant_color)
                # split colors by , to get individual colors
                dominant_color = dominant_color.split(",")
                for color in dominant_color:
                    # Copy the image in the folder "../grouped_by_dominant_color"
                    os.system(f"cp ../output/images/{image} ../output/grouped_by_dominant_color/{color}/{image}")

In [ ]:
# Get all metadata and group by similar metadata

In [ ]:
for metadata in df["metadata"].unique():
    # Metadata are in string formated as key:value,key:value
    # Split the metadata by , to get individual metadata
    metadata = metadata.split(",")
    for meta in metadata:
        try:
            # Split the metadata by : to get the key and the value
            key, value = meta.split(":")
            if not os.path.exists(f"../output/grouped_by_metadata/{key}/{value}"):
                os.makedirs(f"../output/grouped_by_metadata/{key}/{value}")
        except:
            continue

# For each image in the folder "../images_labelized" get the line that contains the dominant color
for image in tqdm(images, desc="Grouping images by metadata"):
    # Get the metadata
    with open("../output/metadata/" + image.split(".")[0] + ".pickle", "rb") as f:
        metadata = pickle.load(f)
        # Convert metadata to string : key:value,key:value
        metadata = ",".join([f"{key}:{value}" for key, value in metadata.items()])
        # Split the metadata by , to get individual metadata
        metadata = metadata.split(",")
        for meta in metadata:
            try:
                # Split the metadata by : to get the key and the value
                key, value = meta.split(":")
            except:
                continue
            # Copy the image in the folder "../grouped_by_metadata"
            os.system(f"cp ../output/images/{image} ../output/grouped_by_metadata/{key}/{value}/{image}")